![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/2c54d005f76747fe83f77fbf8b3ec232).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble._forest import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [ ]:
# Definición de variables predictoras (X)
vec = CountVectorizer(tokenizer=lambda x: x.split(), max_features=1000)
X_dtm = vec.fit_transform(dataTraining['plot'])
X_dtm.shape

In [ ]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [ ]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

print(X_train.shape, X_test.shape, y_train_genres.shape, y_test_genres.shape)

In [ ]:
# Definición y entrenamiento con RandomForestClassifier
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=1000, max_depth=15, random_state=42))

# clf.fit(X_train, y_train_genres)
#radom search
from sklearn.model_selection import RandomizedSearchCV
clf_rd_search = RandomizedSearchCV(estimator=clf,
                                   param_distributions={"estimator__n_estimators": [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
                                                        "estimator__max_depth": [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]},
                                                        n_iter=10, cv=5, verbose=2, random_state=42, n_jobs=-1)
clf_rd_search.fit(X_train, y_train_genres)

# Predicción del modelo de clasificación
y_pred_genres = clf_rd_search.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

# transformación variables predictoras X del conjunto de test
X_test_dtm = vec.transform(dataTesting['plot'])



cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family', 'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance', 'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf_rd_search.predict_proba(X_test_dtm)

# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_RF.csv', index_label='ID')
res.head()

## Guardar el Modelo Calibrado

In [ ]:
# Exportar modelo a archivo binario .pkl
import joblib

joblib.dump(clf_rd_search, 'modelo_predice_generos.pkl', compress=3)

### Ejemplo de ejecución

In [ ]:
# Importar modelo y predicción
from sklearn.feature_extraction.text import CountVectorizer
from predice_generos import predice_GENEROS

corpus=['most is the story of a single father who takes his eight year - old son to work with him at the railroad drawbridge where he is the bridge tender .  a day before ,  the boy meets a woman boarding a train ,  a drug abuser .  at the bridge ,  the father goes into the engine room ,  and tells his son to stay at the edge of the nearby lake .  a ship comes ,  and the bridge is lifted .  though it is supposed to arrive an hour later ,  the train happens to arrive .  the son sees this ,  and tries to warn his father ,  who is not able to see this .  just as the oncoming train approaches ,  his son falls into the drawbridge gear works while attempting to lower the bridge ,  leaving the father with a horrific choice .  the father then lowers the bridge ,  the gears crushing the boy .  the people in the train are completely oblivious to the fact a boy died trying to save them ,  other than the drug addict woman ,  who happened to look out her train window .  the movie ends ,  with the man wandering a new city ,  and meets the woman ,  no longer a drug addict ,  holding a small baby .  other relevant narratives run in parallel ,  namely one of the female drug - addict ,  and they all meet at the climax of this tumultuous film .']

# Predicción de genero
y_pred_test_genres=predice_GENEROS(corpus) 

y_pred_test_genres

## Disponibilizar modelo con Flask

Para esta sección del notebook instale las siguientes librerías *!pip install flask* y *!pip install flask_restplus*.

In [ ]:
import joblib
import werkzeug
from werkzeug.utils import cached_property
from flask import Flask

try:
    from flask_restplus import Api, Resource, fields, marshal
except ImportError:
    import werkzeug
    werkzeug.cached_property = werkzeug.utils.cached_property
    from flask_restplus import Resource, Api, fields, marshal

from predice_generos import predice_GENEROS

# Definición aplicación Flask
app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='API de Predicción de Generos de Peliculas',
    description='API de Predicción de Generos de Peliculas')

ns = api.namespace('predict', 
     description='Predición de Generos de Peliculas')

# Definición argumentos o parámetros de la API
parser = api.parser()



parser.add_argument(
    'Plot', 
    type=str, 
    required=True, 
    help='', 
    location='args')


resource_fields = api.model('Resource', {
    'p_Action': fields.String,
    'p_Adventure' : fields.String,
    'p_Animation' : fields.String,
    'p_Biography' : fields.String,
    'p_Comedy' : fields.String,
    'p_Crime' : fields.String,
    'p_Documentary' : fields.String,
    'p_Drama' : fields.String,
    'p_Family' : fields.String,
    'p_Fantasy' : fields.String,       
    'p_Film-Noir' : fields.String,
    'p_History' : fields.String,
    'p_Horror' : fields.String,
    'p_Music' : fields.String,
    'p_Mystery' : fields.String,
    'p_News' : fields.String,
    'p_Romance' : fields.String,
    'p_Sci-Fi' : fields.String,
    'p_Short' : fields.String,
    'p_Sport' : fields.String,
    'p_Thriller' : fields.String,
    'p_War' : fields.String,
    'p_Western' : fields.String
    })


# Definición de la clase para disponibilización
@ns.route('/')
class GenerosApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        print(args)
        generos=predice_GENEROS([args['Plot']])
        
        return {
        'p_Action': generos['p_Action'],
        'p_Adventure' : generos['p_Adventure'],
        'p_Animation' : generos['p_Animation'],
        'p_Biography' : generos['p_Biography'],
        'p_Comedy' : generos['p_Comedy'],
        'p_Crime' : generos['p_Crime'],
        'p_Documentary' : generos['p_Documentary'],
        'p_Drama' : generos['p_Drama'],
        'p_Family' : generos['p_Family'],
        'p_Fantasy' : generos['p_Fantasy'],       ''
        'p_Film-Noir' : generos['p_Film-Noir'],
        'p_History' : generos['p_History'],
        'p_Horror' : generos['p_Horror'],
        'p_Music' : generos['p_Music'],
        'p_Mystery' : generos['p_Mystery'],
        'p_News' : generos['p_News'],
        'p_Romance' : generos['p_Romance'],
        'p_Sci-Fi' : generos['p_Sci-Fi'],
        'p_Short' : generos['p_Short'],
        'p_Sport' : generos['p_Sport'],
        'p_Thriller' : generos['p_Thriller'],
        'p_War' : generos['p_War'],
        'p_Western' : generos['p_Western']
        }, 200

# Ejecución de la aplicación que disponibiliza el modelo de manera local en el puerto 5000
app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5002)